In [8]:
import requests
import os
import re
import sys
import json
import requests
import argparse
import time
import codecs
from bs4 import BeautifulSoup
from six import u

In [9]:
def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response


In [10]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author, link):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author, link))

In [85]:
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text
    }

    try:
        # 正常狀況取得資料
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']

    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            match_link = re.search('\[(\w*)\]', meta['title']) 
            if match_author:
                meta['author'] = match_author.group(1)
            if match_link:
                meta['link'] = match_link.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('-',meta['author']):
            meta['link'] = '-'
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    #return meta

In [12]:
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries

In [13]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
resp = fetch(url)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2

for entry in post_entries:
    meta = parse_article_meta(entry)
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  

    # pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

TypeError: pretty_print() missing 1 required positional argument: 'link'

In [16]:
import re
import urllib.parse
## from utils import pretty_print

def get_metadata_from(url):

    def parse_next_link(doc):
        domain = 'https://www.ptt.cc/' 
        html = HTML(html=doc)
        controls = html.find('.action-bar a.btn.wide')
        link = controls[1].attrs.get('href')
        return urllib.parse.urljoin(domain, link)

    resp = fetch(url)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    return metadata, next_link

def get_paged_meta(url, num_pages):
    collected_meta = []
    domain = 'https://www.ptt.cc/'
    
    for _ in range(num_pages):
        posts, link = get_metadata_from(url)
        collected_meta += posts

        url = urllib.parse.urljoin(domain, link)
    return collected_meta

#metadata = get_paged_meta(start_url, num_pages=10)
for meta in metadata:
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'],meta['link'])

NameError: name 'metadata' is not defined

In [80]:
def get_posts(metadata):
    # 將所有文章連結收集並串接成完整 URL
    post_links = [
        urllib.parse.urljoin(domain, meta['link'])
        for meta in metadata if 'link' in meta
    ]
    with Pool(processes=8) as pool:
        contents = pool.map(fetch, post_links)
        return contents
    

In [18]:
import time
from multiprocessing import Pool

domain = 'https://www.ptt.cc/'

In [83]:
def parse_article_content(doc,href):
    print(href)
    if '.html' in (href):
        html = HTML(html=doc)
        post_content = html.find('#main-content', first=True).text
        soup = BeautifulSoup(doc, 'html.parser')
        main_content = soup.find(id="main-content")
        PTT_URL = 'https://www.ptt.cc'
        # ex. link would be <a href="/bbs/PublicServan/M.1127742013.A.240.html">Re: [問題] 職等</a>
        link = PTT_URL + href
        article_id = re.sub('\.html', '', href.split('/')[-1])        
        pushes = main_content.find_all('div', class_='push')

        for push in pushes:
            push.extract() ## 移除指定該變數
        filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
        expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        filtered = [_f for _f in filtered if _f]  # remove empty strings
        filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
        content = ' '.join(filtered)
        content = re.sub(r'(\s)+', ' ', content)
        meta = {
            'content':content,
            'article_id':article_id
        }
        return meta
    else:
        meta = {
            'content':'被刪除',
            'article_id': '無'            
        }
        return meta

In [82]:
metadata

[{'title': '[情報] 楊佈新作品 疾風魅影-黑貓中隊',
  'push': '',
  'date': '9/05',
  'author': 'flyover01',
  'link': '/bbs/movie/M.1536146836.A.D69.html'},
 {'title': '[好無雷] 獨家腥聞 (短文)',
  'push': '1',
  'date': '9/05',
  'author': 'frankhood',
  'link': '/bbs/movie/M.1536147157.A.F0C.html'},
 {'title': '[新聞] 「瘋狂亞洲富豪」票房破30億 王力宏包場',
  'push': '6',
  'date': '9/05',
  'author': 'lpbrother',
  'link': '/bbs/movie/M.1536147446.A.CD6.html'},
 {'title': '[新聞] 吳謹言耍大牌 電影大片被祺嬪搶了',
  'push': '9',
  'date': '9/05',
  'author': 'orz44444',
  'link': '/bbs/movie/M.1536147733.A.369.html'},
 {'title': '(本文已被刪除) [james2233]',
  'push': '',
  'date': '9/05',
  'author': 'james2233',
  'link': 'james2233'},
 {'title': '[心得] 這是我認為最接近正義的樣子-聖鹿之死',
  'push': '2',
  'date': '9/05',
  'author': 'james2233',
  'link': '/bbs/movie/M.1536151186.A.228.html'},
 {'title': '[討論] 有參加過特別的首映嗎？',
  'push': '7',
  'date': '9/05',
  'author': 'joey0602',
  'link': '/bbs/movie/M.1536151219.A.BE6.html'},
 {'title': '[超好雷]引爆點：最後引爆的又是甚麼？',
 

In [86]:
if __name__ == '__main__':
    pages = 50
    start_url = 'https://www.ptt.cc/bbs/movie/index.html'
    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages)
    resps = get_posts(metadata)
    
    contents = [parse_article_content(content.text, metadata['link']) for content, metadata in zip(resps, metadata)]

    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    ##for post, resps, text in zip(metadata, resps,contents):
    ##    print('{0} {1: <15} {2}, 網頁內容共 {3} 字, {4}'.format(
    ##        post['date'], post['author'], post['title'], len(resps.text), text['content']))

/bbs/movie/M.1536146836.A.D69.html
/bbs/movie/M.1536147157.A.F0C.html
/bbs/movie/M.1536147446.A.CD6.html
/bbs/movie/M.1536147733.A.369.html
james2233
/bbs/movie/M.1536151186.A.228.html
/bbs/movie/M.1536151219.A.BE6.html
/bbs/movie/M.1536152293.A.CD9.html
/bbs/movie/M.1536156679.A.FA5.html
/bbs/movie/M.1536158237.A.D11.html
/bbs/movie/M.1536160133.A.F59.html
/bbs/movie/M.1536161565.A.DC2.html
/bbs/movie/M.1536162007.A.6DC.html
/bbs/movie/M.1536163643.A.916.html
/bbs/movie/M.1536163693.A.6E3.html
/bbs/movie/M.1536164988.A.60A.html
/bbs/movie/M.1535458622.A.135.html
/bbs/movie/M.1535458800.A.A5B.html
/bbs/movie/M.1535805002.A.1A4.html
/bbs/movie/M.1536114223.A.077.html
/bbs/movie/M.1536114584.A.635.html
/bbs/movie/M.1536117262.A.369.html
/bbs/movie/M.1536118745.A.8D8.html
/bbs/movie/M.1536124186.A.125.html
yaaaa
/bbs/movie/M.1536128976.A.3AE.html
/bbs/movie/M.1536130861.A.939.html
/bbs/movie/M.1536131574.A.33A.html
/bbs/movie/M.1536133152.A.8B0.html
/bbs/movie/M.1536134046.A.2FE.html
/bbs

/bbs/movie/M.1535785709.A.30A.html
/bbs/movie/M.1535787802.A.9B9.html
/bbs/movie/M.1535788444.A.7B9.html
/bbs/movie/M.1535790347.A.A24.html
/bbs/movie/M.1535791743.A.0F9.html
/bbs/movie/M.1535793273.A.404.html
/bbs/movie/M.1535793926.A.DEB.html
/bbs/movie/M.1535795733.A.8D3.html
/bbs/movie/M.1535720931.A.490.html
/bbs/movie/M.1535721641.A.5AD.html
/bbs/movie/M.1535721819.A.724.html
/bbs/movie/M.1535724812.A.CFF.html
/bbs/movie/M.1535726860.A.5C9.html
/bbs/movie/M.1535730035.A.190.html
/bbs/movie/M.1535730134.A.933.html
/bbs/movie/M.1535731792.A.37E.html
/bbs/movie/M.1535732427.A.88C.html
/bbs/movie/M.1535732779.A.08C.html
/bbs/movie/M.1535735407.A.A3B.html
/bbs/movie/M.1535737177.A.773.html
/bbs/movie/M.1535741767.A.EB5.html
/bbs/movie/M.1535756145.A.88B.html
/bbs/movie/M.1535756375.A.05F.html
/bbs/movie/M.1535759098.A.AE9.html
/bbs/movie/M.1535759212.A.AAD.html
/bbs/movie/M.1535760879.A.9E2.html
/bbs/movie/M.1535763599.A.CAC.html
/bbs/movie/M.1535763812.A.7CC.html
/bbs/movie/M.1535701

In [75]:
contents[0]

{'content': '作者 tryzji (奇蹟式.) 看板 movie 標題 [情報] iTunes多部90特價延長 / 新增2部 時間 Mon Sep 3 15:07:39 2018 剛剛看到上週iTunes90元特價的片單，好像因為週末價格設定出問題，活動再延長兩天， 再增加兩部片單特價 漫漫回家路 再見瓦城 前者奧斯卡入圍多項，很愛魯妮瑪拉跟劇情感動大推後者入圍多項金馬獎，聽說也是很 不錯看 目前總共12部片子，都在90元起特價中~有興趣的影迷趕緊出手了 :D 以下節錄自原貼文特價連結 : https://goo.gl/XQPpjh https://goo.gl/H5vSqs 90 摩天樓 : https://goo.gl/hvGLw2 90 倒帶人生 : https://goo.gl/TLwpaH 90 驗屍官 : https://goo.gl/dqmpeZ 90 信長協奏曲 : https://goo.gl/3pSkbu 90 梭哈人生 : https://goo.gl/4dKcu4 90 深夜食堂 : https://goo.gl/3N8YdQ 90 北京遇上西雅圖之不二情書 : https://goo.gl/XL6Tu6 150 千年女優 : https://goo.gl/75GfWQ 150 新天堂樂園 : https://goo.gl/9xrezR 150 再見瓦城 : https://goo.gl/4a1PLB 150 漫漫回家路 : https://goo.gl/wWbhzW',
 'article_id': 'M.1535958469.A.025'}

In [15]:
zip(metadata , contents)

NameError: name 'metadata' is not defined

In [76]:
z

{'title': '[選片] 命帶追逐 引爆點 小偷家族',
 'push': '5',
 'date': '9/03',
 'author': 'stepaside',
 'link': '/bbs/movie/M.1535958298.A.313.html',
 'content': '作者 stepaside (紐牛) 看板 movie 標題 [選片] 命帶追逐 引爆點 小偷家族 時間 Mon Sep 3 15:04:56 2018 手機排版，跪求不要噓我嗚嗚 三部都是近期內有些心動的片 不知道大家比較推哪一部呢？ 身邊較多人看過小偷家族， 似乎比較沉重呢。 命帶追逐預告片看起來ㄎㄧㄤㄎㄧㄤ的， 不曉得會不會其實內容不有趣。 引爆點是衝著吳慷仁的演技， 可是版上滿多負評的。 休假想出門看電影， 請求各位推薦了~~~~ 感謝感謝 Sent from JPTT on my Samsung SMJ730GM. ',
 'article_id': 'M.1535958298.A.313',
 '_id': ObjectId('5b8cf14e360a3e15d49824f3')}

In [122]:
filename = './crawl_file/'+ test['article_id']+'.json'
with open(filename, 'w') as outfile:
    json.dump(test, outfile, ensure_ascii=False)

### 匯入MongoDB

In [50]:
import pymongo
from pymongo import MongoClient


In [51]:
client = pymongo.MongoClient(host='localhost', port=27017)

In [52]:
db = client.ptt

In [53]:
collection = db['article']

In [54]:
student = {
    'id': '20170101',
    'name': 'Jordan',
    'age': 20,
    'gender': 'male'
}

In [55]:
type(student)

dict

In [56]:
type(z)

dict

In [57]:
result = collection.insert([z])


/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


DuplicateKeyError: E11000 duplicate key error collection: ptt.article index: _id_ dup key: { : ObjectId('5b8cedb0360a3e15d49824d9') }

In [58]:
result = collection.find({}, {'title':1})
print(type(result))
print(result[0])

<class 'pymongo.cursor.Cursor'>
{'_id': ObjectId('5b8cedb0360a3e15d49824d9'), 'title': '[選片] 命帶追逐 引爆點 小偷家族'}


In [60]:
for x, y in zip(metadata, contents):
    z = {**x, **y} 
    result = collection.insert([z])    
    #filename = './crawl_file/'+ z['article_id']+'.json'
    #with open(filename, 'w') as outfile:
        #json.dump(z, outfile, ensure_ascii=False)

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
result = collection.find({}, {'content':1, '_id':0})
print(type(result))


<class 'pymongo.cursor.Cursor'>


In [72]:
for i in result:
    print(i)

{'content': '作者 stepaside (紐牛) 看板 movie 標題 [選片] 命帶追逐 引爆點 小偷家族 時間 Mon Sep 3 15:04:56 2018 手機排版，跪求不要噓我嗚嗚 三部都是近期內有些心動的片 不知道大家比較推哪一部呢？ 身邊較多人看過小偷家族， 似乎比較沉重呢。 命帶追逐預告片看起來ㄎㄧㄤㄎㄧㄤ的， 不曉得會不會其實內容不有趣。 引爆點是衝著吳慷仁的演技， 可是版上滿多負評的。 休假想出門看電影， 請求各位推薦了~~~~ 感謝感謝 Sent from JPTT on my Samsung SMJ730GM. '}
{'content': '作者 tryzji (奇蹟式.) 看板 movie 標題 [情報] iTunes多部90特價延長 / 新增2部 時間 Mon Sep 3 15:07:39 2018 剛剛看到上週iTunes90元特價的片單，好像因為週末價格設定出問題，活動再延長兩天， 再增加兩部片單特價 漫漫回家路 再見瓦城 前者奧斯卡入圍多項，很愛魯妮瑪拉跟劇情感動大推後者入圍多項金馬獎，聽說也是很 不錯看 目前總共12部片子，都在90元起特價中~有興趣的影迷趕緊出手了 :D 以下節錄自原貼文特價連結 : https://goo.gl/XQPpjh https://goo.gl/H5vSqs 90 摩天樓 : https://goo.gl/hvGLw2 90 倒帶人生 : https://goo.gl/TLwpaH 90 驗屍官 : https://goo.gl/dqmpeZ 90 信長協奏曲 : https://goo.gl/3pSkbu 90 梭哈人生 : https://goo.gl/4dKcu4 90 深夜食堂 : https://goo.gl/3N8YdQ 90 北京遇上西雅圖之不二情書 : https://goo.gl/XL6Tu6 150 千年女優 : https://goo.gl/75GfWQ 150 新天堂樂園 : https://goo.gl/9xrezR 150 再見瓦城 : https://goo.gl/4a1PLB 150 漫漫回家路 : https://goo.gl/wWbhzW'}
{'content': '作者 qpr322 (龍魚) 看板 movie 標題 [新聞] 賈樟柯對話法國女演